# Computing the Superfluid Stiffness

As I implemented the computation of the superfluid stiffness, I found it rather convenient to be able to compute the superfluid stiffness on my work computer as to not retransfer the data back to the supercomputer. So here you go. Executing this cell will compute all the superfluid stiffness not already computed in *folder*. 
* This uses the compute_all_stiffness easy python script. This method is directly inspired by the program written by Patrick Sémon. It implements exaclty the same code as the stiffness.cpp file in the Autocoherence folder.
* If you wish to change the way the self energy is loaded into the class, please refer to the script/constructG.py file.

**Be careful that the quantity computed here is not the superfluid stiffness but $\beta*\mbox{stiffness}$** ($\beta$ being the inverse temperature)

In [6]:
import os
from importlib import reload  
from scripts import compute_all_stiffness as cp
folder = "./AllData/"
for i,j,y in os.walk(folder):
    if os.path.isdir(os.path.join(i,"DATA")): 
        cp.last_convergedFor_in_folder(i,15)

/home/kown2801/Documents/Programmes/ImpuritySolver/Corrected Versions/Supra/Results/scripts/compute_all_stiffness.py:13: UserWarning: loadtxt: Empty input file: "./AllData/Anti_ferro/Hysteresis/9/ep9_beta60_mu11.45_U12_tpd1.5_tppp1/DATA/stiffness.dat"
  lst = np.sort(np.loadtxt(os.path.join(data_dir,"stiffness.dat")).reshape(-1,2)[:,0].astype(int))
N/A% (0 of 15) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--/home/kown2801/Documents/Programmes/ImpuritySolver/Corrected Versions/Supra/Results/scripts/compute_all_stiffness.py:42: UserWarning: loadtxt: Empty input file: "./AllData/Anti_ferro/Hysteresis/9/ep9_beta60_mu11.45_U12_tpd1.5_tppp1/DATA/stiffness.dat"
  stiffness_data = np.loadtxt(stiffness_filename).reshape(-1,2)
  6% (1 of 15) |#                        | Elapsed Time: 0:00:00 ETA:   0:00:02

./AllData/Anti_ferro/Hysteresis/9/ep9_beta60_mu11.45_U12_tpd1.5_tppp1


100% (15 of 15) |########################| Elapsed Time: 0:00:03 Time:  0:00:03


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
import numpy as np
import constructG as c
import integrator

selfarray_up = np.load("Sid_data/self_energies_mu_up.npy")
selfarray_down = np.load("Sid_data/self_energies_mu_down.npy")

mus_up = [11.35 + 0.01*i for i in range(selfarray_up.shape[0])]
mus_down = [11.57 + 0.01*i for i in range(selfarray_down.shape[0])]
params = {"tpd":1.5, "ep":9,"tpp":1, "tppp":1}

timestart = np.datetime64('now')
stiffness_filename = "DATA/stiffness.dat"

print("Debut time : " + str( np.datetime64('now') - timestart))

def compute_stiffness(params,mu,selfarray):
    integrand = c.Sid_Integrand(mu,\
                            params["tpd"],\
                            params["ep"],\
                            params["tpp"],\
                            params["tppp"],\
                            selfarray)
    # Debut du decompte du temps
    error = 1e-2
    min_value = 1e-5
    stiffness = 0
    last_stiffness = 0
    z = 0
    while z < integrand.getLen() and (z == 0 or stiffness == 0 or last_stiffness/stiffness*(integrand.getLen() - z) > error/10) and (z==0 or stiffness>min_value) :
        integrand.setZ(z)
        last_stiffness = integrator.easy_integrate(integrand.getStifnessFunction(True),np.pi,np.pi)[0]
        stiffness += last_stiffness
        z+=1
        print(z)
    return stiffness

try:
    all_computed_up = np.loadtxt(stiffness_filename + "_up").reshape(-1,3)[:,0]
except Exception as e:
    print(e)
    all_computed_up = []
try:
    all_computed_down = np.loadtxt(stiffness_filename + "_down").reshape(-1,3)[:,0]
except Exception as e:
    print(e)
    all_computed_down = []


def test_and_compute(all_computed,params,mu,selfarray_mu,suffix):
    print(all_computed,mu)
    if not np.isclose(all_computed, mu).any():
        stiffness = compute_stiffness(params,mu,selfarray_mu)
        new_stiffness_data = np.array([mu,stiffness,0]).reshape(-1,3)
        try:
            stiffness_data = np.loadtxt(stiffness_filename + suffix).reshape(-1,3)
            stiffness_data = np.append(stiffness_data,new_stiffness_data,axis=0)
        except Exception as e:
            print(e)
            stiffness_data = new_stiffness_data  
        np.savetxt(stiffness_filename + suffix,stiffness_data,fmt='%1.2f %f %f')

for i in range(len(mus_up)):
    test_and_compute(all_computed_up,params,mus_up[i],selfarray_up[i],"_up")
for i in range(len(mus_down)):
    test_and_compute(all_computed_down,params,mus_down[i],selfarray_down[i],"_down")
    
print("Total time : " + str( np.datetime64('now') - timestart))

In [ ]:
import numpy as np
from importlib import reload  
import constructG as c
import integrator
c = reload(c)

selfarray_up = np.load("Sid_data/self_energies_mu_up.npy")
selfarray_down = np.load("Sid_data/self_energies_mu_down.npy")

mus_up = [11.35 + 0.01*i for i in range(selfarray_up.shape[0])]
mus_down = [11.57 + 0.01*i for i in range(selfarray_down.shape[0])]
params = {"tpd":1.5, "ep":9,"tpp":1, "tppp":1}

timestart = np.datetime64('now')
occupation_filename = "DATA/occupation.dat"

print("Debut time : " + str( np.datetime64('now') - timestart))

def compute_occupation(params,mu,selfarray):
    """
    integrand = c.Sid_Integrand(mu,\
                            params["tpd"],\
                            params["ep"],\
                            params["tpp"],\
                            params["tppp"],\
                            selfarray)
                            """
    integrand = c.Patrick_Integrand(11.45,\
                            params["tpd"],\
                            params["ep"],\
                            params["tpp"],\
                            params["tppp"],
                                    60,
                                   "../Données/Supra/U_12/ep9/60/ep9.0_beta60.0_mu11.45_U12.0/DATA/self11.dat")
    # Debut du decompte du temps
    error = 1e-2
    min_value = 1e-5
    occupation = 0
    last_occupation = 0
    z = 1
    while z < integrand.getLen():
        integrand.setZ(z)
        last_occupation = integrator.integrate(integrand.getOccupation(),np.pi,np.pi,4,12,error)[0]
        occupation += last_occupation
        z+=1
        print(z)
    return occupation

try:
    all_computed_up = np.loadtxt(occupation_filename + "_up").reshape(-1,3)[:,0]
except Exception as e:
    print(e)
    all_computed_up = []
try:
    all_computed_down = np.loadtxt(occupation_filename + "_down").reshape(-1,3)[:,0]
except Exception as e:
    print(e)
    all_computed_down = []


def test_and_compute(all_computed,params,mu,selfarray_mu,suffix):
    print(all_computed,mu)
    if not np.isclose(all_computed, mu).any():
        stiffness = compute_occupation(params,mu,selfarray_mu)
        new_stiffness_data = np.array([mu,stiffness,0]).reshape(-1,3)
        try:
            stiffness_data = np.loadtxt(occupation_filename + suffix).reshape(-1,3)
            stiffness_data = np.append(stiffness_data,new_stiffness_data,axis=0)
        except Exception as e:
            print(e)
            stiffness_data = new_stiffness_data  
        np.savetxt(occupation_filename + suffix,stiffness_data,fmt='%1.2f %f %f')

for i in range(len(mus_up)):
    test_and_compute(all_computed_up,params,mus_up[i],selfarray_up[i],"_up")
for i in range(len(mus_down)):
    test_and_compute(all_computed_down,params,mus_down[i],selfarray_down[i],"_down")
    
print("Total time : " + str( np.datetime64('now') - timestart))

In [ ]:
import numpy as np
a = np.load("Sid_data/self_energies_mu_up.npy")
mus_up = [11.35 + 0.01*i for i in range(len(a))]
mus_down = [11.57 + 0.01*i for i in range(len(a))]
for i,s in enumerate(a):
    c = s.reshape(s.shape[0],64)
    np.savetxt("DATA/self_up_%.2f" % mus_up[i],c.view(float))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
nmax = 50
wmax = 2
beta=(2*nmax+1)*np.pi/wmax
print(beta)
a = np.loadtxt("DATA/stiffness.dat_down")
plt.plot(a[:,0],a[:,1]/beta)
x_total = a[:,0]
Ts = 1/np.array([70,60,50,40,30,20])
a = np.loadtxt("DATA/stiffness.dat_up")
x_total = np.append(x_total,a[:,0])
plt.plot(a[:,0],a[:,1]/beta)
for T in Ts:
    plt.plot(x_total,x_total*0 + T)
plt.show()